# MABe – Reservoir Computing Training

This notebook loads the preprocessed dataset and trains Reservoir Computing models.
It uses the data prepared by `02_dataset_processing_and_scaling.ipynb`.

The goal is twofold:
1. Detect whether an action occurs in a temporal window (action vs no-action).
2. If an action is present, classify the specific behavior (multi-class).

The evaluation is intentionally staged to reflect the structure of the problem
and to handle extreme class imbalance.

In [ ]:
%pip install reservoirpy
%pip install matplotlib
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    average_precision_score,
    precision_recall_curve,
)

from reservoirpy.nodes import Reservoir


## Dataset characteristics

Each sample corresponds to a temporal window extracted from pose-tracking data:
- Shape: (window_size, features)
- window_size = 200 frames
- features = body-part coordinates

Labels:
- A majority of windows correspond to "no action".
- Action windows are extremely rare (≈ 0.4% of the dataset).
- Action classes include: chase, avoid, attack, chaseattack.

This extreme imbalance makes accuracy alone a misleading metric and motivates
the use of precision/recall-based evaluation.

In [ ]:
PROCESSED = Path("data/data_processed")

X = np.load(PROCESSED / "X_windows.npy")  # (N, T, D)
y = np.load(PROCESSED / "y_windows.npy")  # (N,)

# Important: video_id_windows nécessaire pour split propre par vidéo
video_id = np.load(PROCESSED / "video_id_windows.npy", allow_pickle=True)  # (N,)

# Si tu as aussi mouse_id_windows/category_windows, tu peux charger pareil
mouse_id = np.load(PROCESSED / "mouse_id_windows.npy", allow_pickle=True)

print("X:", X.shape, X.dtype)
print("y:", y.shape, y.dtype)
print("video_id:", video_id.shape, video_id.dtype)

assert X.shape[0] == y.shape[0] == video_id.shape[0]


X: (16395, 200, 10) float32
y: (16395,) int64
video_id: (16395,) object


In [16]:
NONE_ID = 0  # adapte si ton mapping est différent

y_binary = (y != NONE_ID).astype(np.int8)  # 1 si action, 0 si none
print("Global action ratio:", y_binary.mean())
print("Total actions:", int(y_binary.sum()), "over", len(y_binary))


Global action ratio: 0.9986581274778896
Total actions: 16373 over 16395


In [17]:
def make_split_with_positives(X, y_binary, video_id=None, test_size=0.2, val_size=0.2, max_tries=200, seed0=42):
    """
    Returns X_train, X_val, X_test, y_train_bin, y_val_bin, y_test_bin, idx_train, idx_val, idx_test
    ensuring val and test contain at least 1 positive sample.
    (Window-level split; quick and robust for debugging.)
    """
    rng = np.random.RandomState(seed0)
    n = len(y_binary)
    idx_all = np.arange(n)

    for t in range(max_tries):
        seed = int(rng.randint(0, 10_000_000))

        idx_train, idx_tmp, y_train, y_tmp = train_test_split(
            idx_all, y_binary, test_size=(test_size + val_size), random_state=seed, stratify=y_binary
        )
        rel_val = val_size / (test_size + val_size)
        idx_val, idx_test, y_val, y_test = train_test_split(
            idx_tmp, y_tmp, test_size=(test_size / (test_size + val_size)), random_state=seed + 1, stratify=y_tmp
        )

        if y_val.sum() > 0 and y_test.sum() > 0:
            return idx_train, idx_val, idx_test

    raise RuntimeError("Could not find a split with positives in both val and test. Need more positives or different strategy.")

idx_train, idx_val, idx_test = make_split_with_positives(X, y_binary, test_size=0.2, val_size=0.2)

X_train, X_val, X_test = X[idx_train], X[idx_val], X[idx_test]
y_train_bin, y_val_bin, y_test_bin = y_binary[idx_train], y_binary[idx_val], y_binary[idx_test]

print("Train:", X_train.shape, "pos:", int(y_train_bin.sum()))
print("Val:  ", X_val.shape,   "pos:", int(y_val_bin.sum()))
print("Test: ", X_test.shape,  "pos:", int(y_test_bin.sum()))


Train: (9837, 200, 10) pos: 9824
Val:   (3279, 200, 10) pos: 3275
Test:  (3279, 200, 10) pos: 3274


In [18]:
def build_reservoir(units=300, sr=0.9, lr=0.3, input_scaling=0.5, seed=42):
    return Reservoir(
        units=units,
        sr=sr,
        lr=lr,
        input_scaling=input_scaling,
        seed=seed
    )

def reservoir_transform(reservoir, X_batch, pooling="last"):
    """
    X_batch: (B, T, D)
    returns Z: (B, units)
    """
    B = X_batch.shape[0]
    Z = np.zeros((B, reservoir.units), dtype=np.float32)

    for i in range(B):
        reservoir.reset()
        states = reservoir.run(X_batch[i])  # (T, units)

        if pooling == "last":
            Z[i] = states[-1]
        elif pooling == "mean":
            Z[i] = states.mean(axis=0)
        else:
            raise ValueError("pooling must be 'last' or 'mean'")

        if (i + 1) % 500 == 0:
            print(f"reservoir_transform {pooling}: {i+1}/{B}")

    return Z

def train_and_eval_stageA(X_train_feat, y_train_bin, X_val_feat, y_val_bin, X_test_feat, y_test_bin, name=""):
    """
    Logistic regression + scaling + choose threshold that maximizes F1 on VAL.
    """
    scaler = StandardScaler(with_mean=True, with_std=True)
    X_train_s = scaler.fit_transform(X_train_feat)
    X_val_s = scaler.transform(X_val_feat)
    X_test_s = scaler.transform(X_test_feat)

    clf = LogisticRegression(
        max_iter=5000,
        class_weight="balanced",
        solver="saga",
        n_jobs=-1,
        random_state=42
    )
    clf.fit(X_train_s, y_train_bin)

    # probs
    p_val = clf.predict_proba(X_val_s)[:, 1]
    p_test = clf.predict_proba(X_test_s)[:, 1]

    # PR-AUC on test (threshold-free)
    pr_auc = average_precision_score(y_test_bin, p_test)

    # threshold sweep on VAL
    precision, recall, thresholds = precision_recall_curve(y_val_bin, p_val)
    # precision_recall_curve returns thresholds of length (len(precision)-1)
    # compute F1 safely for each point
    den = precision + recall
    f1 = np.where(den > 0, 2 * precision * recall / den, 0.0)

    best_idx = int(np.argmax(f1))
    best_thresh = 0.5 if best_idx == len(thresholds) else float(thresholds[best_idx])

    y_pred_test = (p_test >= best_thresh).astype(np.int8)

    cm = confusion_matrix(y_test_bin, y_pred_test, labels=[0, 1])

    print(f"\nStage A ({name})")
    print(f"  Test PR-AUC: {pr_auc:.3f}")
    print(f"  Best threshold (from VAL): {best_thresh:.3f}")
    print("  Confusion matrix [ [TN FP] [FN TP] ]:")
    print(cm)
    print("  Classification report (test):")
    print(classification_report(y_test_bin, y_pred_test, digits=3, zero_division=0))

    return clf, scaler, best_thresh, y_pred_test, p_test


In [19]:
reservoir_params = {'units': 300, 'sr': 0.9, 'lr': 0.3, 'input_scaling': 0.5, 'seed': 42}
reservoir = build_reservoir(**reservoir_params)

# Baseline features (flatten)
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_val_flat   = X_val.reshape(X_val.shape[0], -1)
X_test_flat  = X_test.reshape(X_test.shape[0], -1)

clf_A_base, sc_A_base, th_A_base, yhat_A_base, pA_base = train_and_eval_stageA(
    X_train_flat, y_train_bin, X_val_flat, y_val_bin, X_test_flat, y_test_bin, name="Baseline(flat)"
)

# Reservoir last
Z_train_last = reservoir_transform(reservoir, X_train, pooling="last")
Z_val_last   = reservoir_transform(reservoir, X_val,   pooling="last")
Z_test_last  = reservoir_transform(reservoir, X_test,  pooling="last")

clf_A_last, sc_A_last, th_A_last, yhat_A_last, pA_last = train_and_eval_stageA(
    Z_train_last, y_train_bin, Z_val_last, y_val_bin, Z_test_last, y_test_bin, name="Reservoir(last)"
)

# Reservoir mean
Z_train_mean = reservoir_transform(reservoir, X_train, pooling="mean")
Z_val_mean   = reservoir_transform(reservoir, X_val,   pooling="mean")
Z_test_mean  = reservoir_transform(reservoir, X_test,  pooling="mean")

clf_A_mean, sc_A_mean, th_A_mean, yhat_A_mean, pA_mean = train_and_eval_stageA(
    Z_train_mean, y_train_bin, Z_val_mean, y_val_bin, Z_test_mean, y_test_bin, name="Reservoir(mean)"
)

print("\nAction detection counts in TEST:")
print("  True actions:", int(y_test_bin.sum()))
print("  Pred actions baseline:", int(yhat_A_base.sum()))
print("  Pred actions res-last:", int(yhat_A_last.sum()))
print("  Pred actions res-mean:", int(yhat_A_mean.sum()))



Stage A (Baseline(flat))
  Test PR-AUC: 0.999
  Best threshold (from VAL): 0.000
  Confusion matrix [ [TN FP] [FN TP] ]:
[[   0    5]
 [   0 3274]]
  Classification report (test):
              precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.998     1.000     0.999      3274

    accuracy                          0.998      3279
   macro avg      0.499     0.500     0.500      3279
weighted avg      0.997     0.998     0.998      3279



AttributeError: 'Reservoir' object has no attribute 'state'

In [ ]:
# Stage B ground truth: only true action windows
train_action_mask = (y[idx_train] != NONE_ID)
val_action_mask   = (y[idx_val]   != NONE_ID)
test_action_mask  = (y[idx_test]  != NONE_ID)

X_train_action = X_train[train_action_mask]
X_val_action   = X_val[val_action_mask]
X_test_action  = X_test[test_action_mask]

y_train_action = y[idx_train][train_action_mask]  # labels multi-class but without none
y_val_action   = y[idx_val][val_action_mask]
y_test_action  = y[idx_test][test_action_mask]

print("Stage B datasets (true actions only):")
print("  Train:", X_train_action.shape, "labels:", y_train_action.shape)
print("  Val:  ", X_val_action.shape,   "labels:", y_val_action.shape)
print("  Test: ", X_test_action.shape,  "labels:", y_test_action.shape)

if len(y_test_action) == 0:
    print("\nWARNING: No true action windows in TEST. Stage B cannot be evaluated on this split.")


## Reservoir Computing Model

Define and train the reservoir model.

In [ ]:
def train_and_eval_stageB(X_train_feat, y_train_mc, X_test_feat, y_test_mc, name=""):
    scaler = StandardScaler(with_mean=True, with_std=True)
    X_train_s = scaler.fit_transform(X_train_feat)
    X_test_s = scaler.transform(X_test_feat)

    clf = LogisticRegression(
        max_iter=5000,
        class_weight="balanced",
        solver="saga",
        n_jobs=-1,
        random_state=42,
        multi_class="multinomial"
    )
    clf.fit(X_train_s, y_train_mc)
    y_pred = clf.predict(X_test_s)

    print(f"\nStage B ({name})")
    print("  Classification report (test, true action windows):")
    print(classification_report(y_test_mc, y_pred, digits=3, zero_division=0))
    print("  Confusion matrix:")
    print(confusion_matrix(y_test_mc, y_pred))

    return clf, scaler, y_pred

if len(y_test_action) == 0:
    print("Skipping Stage B due to empty test_action set.")
else:
    # Baseline: flatten
    X_train_action_flat = X_train_action.reshape(X_train_action.shape[0], -1)
    X_test_action_flat  = X_test_action.reshape(X_test_action.shape[0], -1)
    clf_B_base, sc_B_base, yhat_B_base = train_and_eval_stageB(
        X_train_action_flat, y_train_action, X_test_action_flat, y_test_action, name="Baseline(flat)"
    )

    # Reservoir last
    Z_train_action_last = reservoir_transform(reservoir, X_train_action, pooling="last")
    Z_test_action_last  = reservoir_transform(reservoir, X_test_action,  pooling="last")
    clf_B_last, sc_B_last, yhat_B_last = train_and_eval_stageB(
        Z_train_action_last, y_train_action, Z_test_action_last, y_test_action, name="Reservoir(last)"
    )

    # Reservoir mean
    Z_train_action_mean = reservoir_transform(reservoir, X_train_action, pooling="mean")
    Z_test_action_mean  = reservoir_transform(reservoir, X_test_action,  pooling="mean")
    clf_B_mean, sc_B_mean, yhat_B_mean = train_and_eval_stageB(
        Z_train_action_mean, y_train_action, Z_test_action_mean, y_test_action, name="Reservoir(mean)"
    )


Stage A: Baseline


C:\Users\flore\AppData\Local\Temp\ipykernel_32192\2207658381.py:33: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)
c:\Users\flore\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])



Stage A (Baseline):
  PR-AUC: 0.026
  Threshold: 0.609
  Precision: 0.000, Recall: 0.000, F1: 0.000
  Confusion Matrix:
[[5879   61]
 [   0    0]]
  Threshold Sweep (sample):
    0.00: P=0.002, R=1.000, F1=0.004
    0.03: P=0.002, R=1.000, F1=0.004
    0.05: P=0.002, R=0.833, F1=0.004
    0.08: P=0.002, R=0.833, F1=0.005
    0.12: P=0.003, R=0.833, F1=0.005
    0.15: P=0.003, R=0.833, F1=0.007
    0.18: P=0.004, R=0.833, F1=0.008
    0.21: P=0.005, R=0.833, F1=0.011
    0.25: P=0.007, R=0.667, F1=0.013
    0.30: P=0.013, R=0.667, F1=0.025
    1.00: P=0.000, R=0.000, F1=0.000
Stage A: Reservoir (last pooling)


C:\Users\flore\AppData\Local\Temp\ipykernel_32192\2207658381.py:33: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)
c:\Users\flore\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])



Stage A (Reservoir Last):
  PR-AUC: 0.002
  Threshold: 0.346
  Precision: 0.000, Recall: 0.000, F1: 0.000
  Confusion Matrix:
[[3033 2907]
 [   0    0]]
  Threshold Sweep (sample):
    0.00: P=0.002, R=1.000, F1=0.004
    0.04: P=0.002, R=1.000, F1=0.004
    0.08: P=0.002, R=0.833, F1=0.004
    0.21: P=0.002, R=0.833, F1=0.005
    0.33: P=0.003, R=0.833, F1=0.005
    0.42: P=0.001, R=0.333, F1=0.003
    0.48: P=0.002, R=0.333, F1=0.003
    0.57: P=0.002, R=0.333, F1=0.004
    0.65: P=0.002, R=0.167, F1=0.003
    0.72: P=0.000, R=0.000, F1=0.000
    0.96: P=0.000, R=0.000, F1=0.000
Stage A: Reservoir (mean pooling)

Stage A (Reservoir Mean):
  PR-AUC: 0.002
  Threshold: 0.214
  Precision: 0.000, Recall: 0.000, F1: 0.000
  Confusion Matrix:
[[2507 3433]
 [   0    0]]
  Threshold Sweep (sample):
    0.00: P=0.002, R=1.000, F1=0.004
    0.01: P=0.002, R=1.000, F1=0.004
    0.04: P=0.002, R=1.000, F1=0.005
    0.12: P=0.003, R=1.000, F1=0.006
    0.27: P=0.003, R=0.833, F1=0.005
    0.40: 

c:\Users\flore\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\flore\AppData\Local\Temp\ipykernel_32192\2207658381.py:33: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)
c:\Users\flore\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


ValueError: cannot reshape array of size 0 into shape (0,newaxis)